# Лабораторная работа 1. Подсчет ошибки распознавания

WER (Word Error Rate) является метрикой для оценки качества систем распознавания речи. Она показывает процент ошибочных слов в гипотезе распознавания по сравнению с эталонным текстом. WER учитывает три типа ошибок: вставки, удаления и замены. 
$$ WER = {I + D + S \over D + S + C} $$
где I, D, S - количество втавок, удалений и замен, соответственно. C - количество правильно распознанных слов

Лабораторная работа состоит из трех частей. Первая часть (функция подсчета WER) обязательная, остальные дополнительные. Всего за работу можно получить максимум 20 баллов. 4 за сдачу в срок и 16 за задания: 
* функция подсчета WER (тест 1.a, 1.b) - 8 баллов
* функция подсчета WER и ошибки пунктуации (тест 2.a) - 4 балла
* функция подсчета SA-WER (тест 3.а) - 4 балла

# Общая секция &mdash; импорты, инициализация, константы

In [1]:
import re
import itertools

In [2]:
# знаки пунктуации
RE_PUNCTUATIONS = re.compile(r"(?u)([\.!?,;:\-\(\)\"'`«»$%*/\|+~\^&#\{\}\[\]=<>\\]+)")

# пробельные символы
RE_WHITESPACE = re.compile(r"\s+")

# 1. Word Error Rate (8 баллов)

## 1.a. подсчет WER 


Функция должна принимать две строки в качестве входных данных: эталонный текст и распознанный текст. Эталонный текст - это то, что произносится в аудиозаписи, а гипотеза распознавания - это текст, полученный от системы распознавания речи. Для корректного вычисления ошибки распознавания необходимо удалить все символы пунктуации и привести все слова к нижнему регистру.



In [3]:
def preprocess_text(src:str, original_case:bool=False) -> list[str]:
    """Удаление знаков пунктуации и приведение к нижнему регистру."""
    result = re.sub(RE_PUNCTUATIONS, "", src)

    if not original_case:
        result = result.lower()

    result = re.split(RE_WHITESPACE, result)
    result = [ word.strip() for word in result if word.strip() != "" ]

    return result

In [4]:
def creade_distance_matrix(reference_text:str, recognized_text:str) -> list[list[int]]:
    """Построение матрицы расстояний."""

    # Приведение текста к нижнему регистру, удаление символов пунктуации и разбивка на слова
    reference_words = preprocess_text(reference_text)
    recognized_words = preprocess_text(recognized_text)

    # Инициализация матрицы для подсчета расстояния между словами
    distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]
    
    # Наполнение первой строки матрицы
    for i in range(len(reference_words) + 1):
        distance_matrix[i][0] = i

    # Наполнение первого столбца матрицы
    for j in range(len(recognized_words) + 1):
        distance_matrix[0][j] = j

    def cost(i:int, j:int) -> int:
        return 0 if reference_words[i-1] == recognized_words[j-1] else 1

    # Заполнение матрицы расстояний методом динамического программирования
    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(recognized_words) + 1):
            distance_matrix[i][j] = min([
                distance_matrix[i][j-1] + 1,
                distance_matrix[i-1][j] + 1,
                distance_matrix[i-1][j-1] + cost(i, j),
            ])

    return distance_matrix


def calculate_wer(reference_text: str, recognized_text: str) -> float:
    """Вычисление WER."""

    # строим матрицу расстояний
    distance_matrix = creade_distance_matrix(reference_text, recognized_text)

    # расстояние Левенштейна - самый правый нижний элемент
    levenstein_distance = distance_matrix[-1][-1]

    # Расчет WER  (в процентах)
    wer = levenstein_distance / (len(distance_matrix) - 1) * 100

    return wer

# отладочный запуск
wer = calculate_wer('Я ел солонину', 'Я ел слона')
print(f"Word Error Rate: {wer:.2f}%")

Word Error Rate: 33.33%


In [5]:
def assert_wer(ref, hyp, ideal_wer):
    wer = calculate_wer(ref, hyp)
    assert round(wer, 2) == round(ideal_wer, 2), f"for '{hyp=}' and '{ref=}' {ideal_wer=}, calculate_wer {wer=}"
    
def test_wer():
    assert_wer('привет студент', 'привет студент', 0)
    assert_wer('привет! Студент.', 'Привет, студент?', 0)
    assert_wer('привет студент', 'студент', 50)
    assert_wer('привет студент', '', 100)
    assert_wer('привет студент', 'студент привет', 100)
    assert_wer('привет', 'привет студент', 100)
    assert_wer('привет студент привет как дела', 'студент привет', 60)
    assert_wer('привет студент привет как дела', 'привет как дела', 40)
    assert_wer('привет студент привет как дела ', 'привет студент дела ', 40)
    assert_wer('привет студент привет как дела '*100, 'привет студент дела '*100, 40)

    print(f"Test 1.a passed")
    
test_wer() 
    

Test 1.a passed


## 1.b. Построение выравнивания
Реализованная в части 1.a. функция выдает только суммарное значение ошибки распознавания, не давая понимания, в чем состоят основные проблемы распознавания. 

Значение WER получается из трех видов ошибок: 
* вставка (insertion)
* удаление (deletion)
* замена (substitution)

Каждый тип ошибок имеет свое значение и указывает на определенные недостатки системы. Например, большое количество вставок означает, что ASR слышит речь там где ее нет. А большое количество удалений показывает, что целевая речь пропускается и не транскрибируется. 

Кроме числовых значений каждой ошибки, для анализа результатов работы системы может пригодиться выравнивание эталонной текстовки и гипотезы распознавания относительно друг друга. 

пример выравнивания: 

```
>>> tabulate(ali)

Я сегодня  ***   учуcь  в  универе
Я    с    завтра учусь *** универе  
C    S      I      C    D    C    
```

Реализуйте функцию, которая кроме числового значения WER возвращает выравнивание, а также значения каждого типа ошибок распознавания (вставки, удаления, замены).

In [6]:
!pip install tabulate
from tabulate import tabulate
# используйте tabulate для отладки

Defaulting to user installation because normal site-packages is not writeable


In [7]:
def calculate_wer_with_alignment(reference_text: str, recognized_text: str):

    # Перенесите сюда код из задания 1.a.

    # расстояние Левенштейна
    distance_matrix = creade_distance_matrix(reference_text, recognized_text)

    levenstein_distance = distance_matrix[-1][-1]

    wer = levenstein_distance / (len(distance_matrix) - 1) * 100

    # разбивка текста на слова и удаление пунктуации
    reference_words = preprocess_text(reference_text, original_case=True)
    recognized_words = preprocess_text(recognized_text, original_case=True)

    # используя distance_matrix восстановите путь (набор операций), который соответстует найденому WER 
    insertion = 0     # horisontal move
    deletion = 0      # vertical move
    substitution = 0  # diagonal move
    correct = 0
    ali = [
        [], # reference
        [], # recognized
        [], # operations
    ]

    i = len(distance_matrix) - 1
    j = len(distance_matrix[0]) - 1
    finished = False
    while i > 0 or j > 0:
        current_value = distance_matrix[i][j]

        try_insertion = distance_matrix[i][j-1]
        try_deletion = distance_matrix[i-1][j]
        try_substitution = distance_matrix[i-1][j-1]

        min_value = min([try_insertion, try_deletion, try_substitution])

        # ход по диагонали
        if try_substitution == min_value and abs(current_value - min_value) <= 1:
            if current_value - min_value == 1:
                substitution += 1
                ali[2].append("S")
            else:
                correct += 1
                ali[2].append("C")

            i -= 1
            j -= 1

            ali[0].append(reference_words[i])
            ali[1].append(recognized_words[j])

        # ход по горизонтали
        elif try_insertion == min_value and abs(current_value - min_value) <= 1:
            insertion += 1
            j -= 1

            ali[2].append("I")
            ali[0].append("***")
            ali[1].append(recognized_words[j])


        # ход по вертикали
        else:
            deletion += 1
            i -= 1

            ali[2].append("D")
            ali[0].append(reference_words[i])
            ali[1].append("***")


    # ali[0]=  разбитый по словам референс. Втавки отабражаются в эталонном выравнивании с помощью "***"
    # ali[1] = разбитая по словам гипотеза.
    # ali[2] = аннотация 
    assert len(ali[0]) == len(ali[1]) == len(ali[2]), f"wrong ali {ali}"
    
    return {"wer" : wer,
            "cor": correct,
            "del": deletion,
            "ins": insertion,
            "sub": substitution,
            "ali": [ list(reversed(line)) for line in ali ]}


# отладочный запуск
result = calculate_wer_with_alignment(
    "Я сегодня учусь в университете ИТМО",
    "Я с завтрашнего дня учусь в ИТМО"
)

print(f"""
wer = {result["wer"]}
cor = {result["cor"]}
del = {result["del"]}
ins = {result["ins"]}
sub = {result["sub"]}

Результат:

{tabulate(result["ali"], stralign="center", tablefmt="plain")}
""")

print()


wer = 66.66666666666666
cor = 4
del = 1
ins = 2
sub = 1

Результат:

Я  ***      ***      сегодня  учусь  в  университете  ИТМО
Я   с   завтрашнего    дня    учусь  в      ***       ИТМО
C   I        I          S       C    C       D         C




In [8]:
# def calculate_wer_with_alignment(reference_text: str, recognized_text: str):
#     return {
#             "wer" : 0,
#             "cor": 2, 
#             "del": 0,
#             "ins": 0,
#             "sub": 0,
#             "ali": [["привет", "студент"],["привет", "студент"],['C', 'C']]}

In [9]:
def assert_wer_with_alignment(ref, hyp, ideal_report):
    report = calculate_wer_with_alignment(ref, hyp)
    for k, v in ideal_report.items():
        if isinstance(v, float):
            assert round(v, 2) == round(report[k], 2), f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"
        else:
            assert v == report[k], f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"

    
def test_wer_with_alignment():
    assert_wer_with_alignment('привет студент', 'привет студент',  {
            "wer" : 0,
            "cor": 2, 
            "del": 0,
            "ins": 0,
            "sub": 0,
            "ali": [["привет", "студент"],["привет", "студент"],['C', 'C']]})
    assert_wer_with_alignment('привет студент', 'студент', {
            "wer" : 50,
            "cor": 1, 
            "del": 1,
            "ins": 0,
            "sub": 0,
            "ali": [["привет", "студент"],["***", "студент"],['D', 'C']]})
    assert_wer_with_alignment('привет', 'привет студент', {
            "wer" : 100,
            "cor": 1, 
            "del": 0,
            "ins": 1,
            "sub": 0,
            "ali": [["привет", "***"],["привет", "студент"],['C', 'I']]})
    assert_wer_with_alignment('привет студент', 'пока студент',  {
            "wer" : 50,
            "cor": 1, 
            "del": 0,
            "ins": 0,
            "sub": 1,
            "ali": [["привет", "студент"],["пока", "студент"],['S', 'C']]})

    print(f"Test 1.b passed")
    
test_wer_with_alignment() 

Test 1.b passed


# 2. WER с пунктуацией (4 балла)
Попробуйте модифицировать WER таким образом, чтобы получившаяся метрика учитавала ошибки расстановки знаков препинания. 

Для этого надо ввести ограничение в алгоритм подсчета distance_matrix таким образом, чтобы запретить делать замену знака препинания на слово и наоборот.

Пример выравнивания 
```
Я сегодня  .   ***   ***  А ты  
Я    с    *** завтра  ?   А ты  
C    S    D_p   I    I_p  C  C    
```
Здесь суффикс _p в аннотации к ошибкам означает **ошибки пунктуации**


Задание: 
Напишите функцию, которая кроме стандартного WER считает дополнительно PunctuaionErrorRate (PER) по формуле

$$ PER = {I_p + D_p + S_p \over D_p + S_p + C_p} $$


In [10]:
def preprocess_text_with_punctuation(src:str, original_case:bool=False) -> list[str]:
    """Разбиение текста на слова и значи препинания."""
    result = re.sub(RE_PUNCTUATIONS, r" \1 ", src)

    if not original_case:
        result = result.lower()

    result = re.split(RE_WHITESPACE, result)
    result = [ word.strip() for word in result if word.strip() != "" ]

    return result

def is_word(str) -> bool:
    """Проверка, является ли слово словом или знаком препинания."""
    return not RE_PUNCTUATIONS.match(str)

def same(str1, str2):
    """Возвращает True, или если обе строки являются словами, 
    или если обе спроки являются знаками препинания."""

    return not (is_word(str1) ^ is_word(str2))

# отладочный запуск
str = "Я сегодня. А ты?"
tokens = preprocess_text_with_punctuation(str, original_case=True)
print("Токенизированная строка:", tokens)

print()
for token in tokens:
    print(f"{token} -> {'слово' if is_word(token) else 'знак пунктуации'}")

print()
print(f"слово1 <> ?      =", same("слово1", "?"))
print(f"слово1 <> слово2 =", same("слово1", "слово2"))
print(f"!      <> слово2 =", same("!",      "слово2"))
print(f"!      <> ?      =", same("!",      "?"))

Токенизированная строка: ['Я', 'сегодня', '.', 'А', 'ты', '?']

Я -> слово
сегодня -> слово
. -> знак пунктуации
А -> слово
ты -> слово
? -> знак пунктуации

слово1 <> ?      = False
слово1 <> слово2 = True
!      <> слово2 = False
!      <> ?      = True


In [11]:
def creade_distance_matrix_with_punctuation(reference_text: str, recognized_text: str) -> list[list[int]]:
    """Построение матрицы расстояний с учетом пунктуации."""

    # Приведение текста к нижнему регистру, удаление символов пунктуации и разбивка на слова
    reference_words = preprocess_text_with_punctuation(reference_text)
    recognized_words = preprocess_text_with_punctuation(recognized_text)

    # Инициализация матрицы для подсчета расстояния между словами
    distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]
    
    # Наполнение первой строки матрицы
    for i in range(len(reference_words) + 1):
        distance_matrix[i][0] = i

    # Наполнение первого столбца матрицы
    for j in range(len(recognized_words) + 1):
        distance_matrix[0][j] = j

    def cost(i:int, j:int) -> int:
        return 0 if reference_words[i-1] == recognized_words[j-1] else 1

    # Заполнение матрицы расстояний методом динамического программирования
    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(recognized_words) + 1):
            operations = [
                distance_matrix[i][j-1] + 1,
                distance_matrix[i-1][j] + 1,
            ]

            if same(reference_words[i-1], recognized_words[j-1]):
                operations.append(distance_matrix[i-1][j-1] + cost(i, j))

            distance_matrix[i][j] = min(operations)

    return distance_matrix

In [12]:
def calculate_wer_per(reference_text: str, recognized_text: str):

    # разбивка текста на слова и удаление пунктуации
    reference_words = preprocess_text(reference_text, original_case=True)

    # разбивка текста с пунктуацией
    reference_words_punkt = preprocess_text_with_punctuation(reference_text, original_case=True)
    recognized_words_punkt = preprocess_text_with_punctuation(recognized_text, original_case=True)

    # используя distance_matrix восстановите путь (набор операций), который соответстует найденому WER 
    insertion = 0     # horisontal move
    deletion = 0      # vertical move
    substitution = 0  # diagonal move
    correct = 0
    ali = [
        [], # reference
        [], # recognized
        [], # operations
    ]

    # матрица расстояний для текста со знаками препинания
    distance_matrix = creade_distance_matrix_with_punctuation(reference_text, recognized_text)

    # количество вставок, удалений и замен для знаков препинания
    levenstein_distance = 0
    levenstein_distance_per = 0

    i = len(distance_matrix) - 1
    j = len(distance_matrix[0]) - 1
    finished = False
    while i > 0 or j > 0:
        current_value = distance_matrix[i][j]

        try_insertion = distance_matrix[i][j-1]
        try_deletion = distance_matrix[i-1][j]
        try_substitution = distance_matrix[i-1][j-1]

        min_value = min([try_insertion, try_deletion, try_substitution])

        # ход по диагонали
        if try_substitution == min_value and abs(current_value - min_value) <= 1:
            if current_value - min_value == 1:
                substitution += 1
                ali[2].append("S")
            else:
                correct += 1
                ali[2].append("C")

            i -= 1
            j -= 1

            ali[0].append(reference_words_punkt[i])
            ali[1].append(recognized_words_punkt[j])

            if not is_word(reference_words_punkt[i]):
                ali[2][-1] += "_p"
                if ali[2][-1] != "C_p":
                    levenstein_distance_per += 1
            else:
                if ali[2][-1] != "C":
                    levenstein_distance += 1

        # ход по вертикали
        elif try_deletion == min_value and abs(current_value - min_value) <= 1:
            deletion += 1
            i -= 1

            ali[2].append("D")
            ali[0].append(reference_words_punkt[i])
            ali[1].append("***")

            if not is_word(reference_words_punkt[i]):
                ali[2][-1] += "_p"
                levenstein_distance_per += 1
            else:
                levenstein_distance += 1

        # ход по горизонтали
        else:
            insertion += 1
            j -= 1

            ali[2].append("I")
            ali[0].append("***")
            ali[1].append(recognized_words_punkt[j])

            if not is_word(recognized_words_punkt[j]):
                ali[2][-1] += "_p"
                levenstein_distance_per += 1
            else:
                levenstein_distance += 1

    # ali[0]=  разбитый по словам референс. Втавки отабражаются в эталонном выравнивании с помощью "***"
    # ali[1] = разбитая по словам гипотеза.
    # ali[2] = аннотация 
    assert len(ali[0]) == len(ali[1]) == len(ali[2]), f"wrong ali {ali}"

    wer = levenstein_distance / (len(reference_words)) * 100
    per = levenstein_distance_per / (len(reference_words_punkt) - len(reference_words)) * 100

    return {"wer": wer, # calculate_wer(reference_text, recognized_text),
            "per": per,
            "ali": [ list(reversed(line)) for line in ali ]}


# отладочный запуск
result = calculate_wer_per(
    "Я сегодня учусь в университете ИТМО. A ты?",
    "А я? С завтрашнего дня учусь в ИТМО."
)

print(f"""
wer = {result["wer"]}
per = {result["per"]}

Результат:

{tabulate(result["ali"], stralign="center", tablefmt="plain")}
""")

print()


wer = 87.5
per = 100.0

Результат:

***  Я  ***  ***      ***      сегодня  учусь  в  университете  ИТМО   .    A   ты    ?
 А   я   ?    С   завтрашнего    дня    учусь  в      ***       ИТМО   .   ***  ***  ***
 I   C  I_p   I        I          S       C    C       D         C    C_p   D    D   D_p




In [13]:
def assert_wer_per(ref, hyp, ideal_report):
    report = calculate_wer_per(ref, hyp)
    for k, v in ideal_report.items():
        if isinstance(v, float):
            assert round(v, 2) == round(report[k], 2), f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"
        else:
            assert v == report[k], f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"

    
def test_wer_per():
    assert_wer_per('привет студент.', 'привет студент',  {
            "wer" : 0,
            "per": 100})
    assert_wer_per('привет студент.', 'студент.', {
            "wer" : 50,
            "per": 0,})
    assert_wer_per('привет студент.', 'привет. студент',  {
            "wer" : 0,
            "per": 200})
    assert_wer_per('привет студент.', '.студент?', {
            "wer" : 50,
            "per": 200, })

    print(f"Test 2 passed")
    
test_wer_per() 

Test 2 passed


# 3. Speaker-attributed Word Error Rate (4 балла)

В задаче распознавания диалоговых данных, когда говорят два диктора, важно не только распознать правильно каждое слово, но и отнести его к правильному диктору. Чаще всего такой результат получается с помощью комбинации двух независимых систем: распознавания речи и диаризация. 

При подсчете ошибки распознавания диалоговых систем в формулу WER добавляется еще один тип ошибки - S_I (speaker incorrect).

$$ SA{\text -}WER = \min{I + D + S + S_I \over D + S + C + S_I} $$

Кроме подсчета самой ошибки, sa-wer решает еще одну задачку - поиск маппинга из эталонных названий спикеров (например, имен) в предсказанные (чаще всего Idшники). Это необходимо, тк система диаризации не знает, какие названия у спикеров в эталоне. При подсчете SA-WER проверяются все возможные мапинги спикеров и выбирается тот, который соответствует минимальному значению ошибки. 



In [14]:
def create_distance_matrix_simplified(reference_words:list[str], recognized_words:list[str]) -> list[list[int]]:
    """Построение матрицы расстояний по массивам слов."""

    # Инициализация матрицы для подсчета расстояния между словами
    distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]
    
    # Наполнение первой строки матрицы
    for i in range(len(reference_words) + 1):
        distance_matrix[i][0] = i

    # Наполнение первого столбца матрицы
    for j in range(len(recognized_words) + 1):
        distance_matrix[0][j] = j

    def cost(i:int, j:int) -> int:
        return 0 if reference_words[i-1] == recognized_words[j-1] else 1

    # Заполнение матрицы расстояний методом динамического программирования
    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(recognized_words) + 1):
            distance_matrix[i][j] = min([
                distance_matrix[i][j-1] + 1,
                distance_matrix[i-1][j] + 1,
                distance_matrix[i-1][j-1] + cost(i, j),
            ])

    return distance_matrix

In [15]:
def calculate_sawer(reference_text:list[str], reference_speakers:list[str], recognized_text:list[int], recognized_speakers:list[str]):

    # В отличие от прошлых функций на вход sawer подаются уже разбитые на слова произнесения
    # Кроме списка слов, дополнительно передается список меток спикеров
    assert isinstance(reference_text, list)
    assert isinstance(recognized_text, list)
    assert len(reference_text) == len(reference_speakers)
    assert len(recognized_text) == len(recognized_speakers)
    
    # TODO  посчитайте sawer с учетом мапинга спикеров
    # для этого посчитайте значение ошибки для каждого варианта мапинга меток дикторов 
    # и выберете тот, который соответствует минимальному SA-WER

    # расстояние Левенштейна
    distance_matrix = create_distance_matrix_simplified(reference_text, recognized_text)
    levenstein_distance = distance_matrix[-1][-1]

    # используя distance_matrix составляем список правильно распознанных слов 
    # и соответствующих им эталонных и распознанных дикторов
    speakers_sequence = []

    i = len(distance_matrix) - 1
    j = len(distance_matrix[0]) - 1
    finished = False
    while i > 0 or j > 0:
        current_value = distance_matrix[i][j]

        try_insertion = distance_matrix[i][j-1]
        try_deletion = distance_matrix[i-1][j]
        try_substitution = distance_matrix[i-1][j-1]

        min_value = min([try_insertion, try_deletion, try_substitution])

        # ход по диагонали
        if try_substitution == min_value and abs(current_value - min_value) <= 1:

            # корректно распознанное слово
            if current_value - min_value != 1:
                speakers_sequence.append({
                    "reference_name": reference_speakers[i-1],
                    "recognized_id": recognized_speakers[j-1],
                })

            i -= 1
            j -= 1

        elif try_insertion == min_value and abs(current_value - min_value) <= 1:
            j -= 1

        else:
            i -= 1

    # т.к. шли с конца, возвращаем верный порядок
    speakers_sequence.reverse()

    #
    # теперь необходимо найти все возможные варианты мапингов `{"reference_name": reference_id}`
    #

    # выбираем унивальные имена дикторов, слова которых распознаны,
    # т.к. только они участвуют в расчетах
    reference_names = list({ speaker["reference_name"] for speaker in speakers_sequence })

    # выбираем уникальные идентификаторы дикторов для слов, которые были распознаны
    recognized_ids = list({ speaker["recognized_id"] for speaker in speakers_sequence })
    if len(recognized_ids) < len(reference_names):
        recognized_ids.extend([None]*(len(reference_names) - len(recognized_ids)))

    # собираем все возможные варианты соответствия
    ids_permutations = list(itertools.permutations(recognized_ids))

    # считаем количество ошибочных распознаваний для каждой комбинации
    # и попутно ищем минимальное
    min_detection_errors = None
    best_mapping = None
    for reference_ids in ids_permutations:
        mapping = { combination[0]: combination[1] for combination in zip(reference_names, reference_ids) }

        detection_errors = 0
        for speaker in speakers_sequence:
            reference_id = mapping[speaker["reference_name"]]
            if reference_id != speaker["recognized_id"]:
                detection_errors += 1
        
        if min_detection_errors is None or detection_errors < min_detection_errors:
            best_mapping = mapping
            min_detection_errors = detection_errors

    sawer = (levenstein_distance + min_detection_errors) / len(reference_text) * 100

    ali = best_mapping
    return {"sawer" : sawer, 
            "ali": ali}

sawer = calculate_sawer(
    reference_text=['привет', 'студент', "как", "дела"],
    recognized_text=['привет', 'студент', "как", "дела"],
    reference_speakers=['А', 'B', "С", "А"],
    recognized_speakers=[1, 2, 1, 1],
)

print(sawer)

{'sawer': 25.0, 'ali': {'А': 1, 'B': 2, 'С': None}}


In [16]:
def assert_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers, ideal_report):
    report = calculate_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers)
    for k, v in ideal_report.items():
        assert v == report[k]

    
def test_sawer():
    assert_sawer(['привет', 'студент'], ['A', 'B'], ['привет', 'студент'], [1, 2],  {
            "sawer" : 0})
    assert_sawer(['привет', 'студент'], ['A', 'A'], ['привет', 'студент'], [1, 2],  {
            "sawer" : 50})
    assert_sawer(['привет', 'студент'], ['A', 'A'], ['привет', 'студент'], [0, 0],  {
            "sawer" : 0})
    assert_sawer(['привет', 'с'], ['A', 'B'], ['привет', 'студент'], [1, 2],  {
            "sawer" : 50})
    assert_sawer(['привет', 'с'], ['A', 'B'], ['привет'], [1],  {
            "sawer" : 50})
    assert_sawer(['привет'], ['A'], ['привет', 'студент'], [1, 0],  {
            "sawer" : 100})
    assert_sawer(['привет'], ['A'], ['привет', 'студент'], [0, 0],  {
            "sawer" : 100})

    print(f"Test 3 passed")
    
test_sawer()

Test 3 passed
